#Graido 목업 앱

### ----- 라이브러리 설치-----


gradio는 streamlit과 기본 문법과 구조가 비슷하므로 해당 부분만 ChatGPT 등을 이용해 Streamlit에 맞게 변환해서 연결하면 됨.

In [1]:
!pip install gradio
!pip install --upgrade gradio
# 설치가 되면 런타임 재시작 후 이 셀 건너뛰고 시작
!pip install PyPortfolioOpt
!pip install yfinance
!pip install matplotlib

In [2]:
import pandas as pd
import numpy as np
import gradio as gr
from pypfopt import EfficientFrontier, risk_models, expected_returns
import yfinance as yf
import matplotlib.pyplot as plt

## 1. 기존 방법 (1차로 종목 선정, 2차로 YF 포트폴리오 라이브러리 적용)

### 데이터셋 불러오기

In [ ]:
dummy = pd.read_csv('/content/drive/MyDrive/Kwargs/240820_final_dummy.csv', encoding='euc-kr')

In [ ]:
dummy

,Company,Year,average_label,industry,environmental,social,governance,ticker
0,KB금융,2019,5.016282,Finance,88,85,62,105560.KS
1,KB금융,2020,5.109671,Finance,78,63,81,105560.KS
2,KB금융,2021,5.996064,Finance,64,80,88,105560.KS
3,KB금융,2022,6.966534,Finance,92,97,98,105560.KS
4,KB금융,2023,7.012989,Finance,57,64,81,105560.KS
...,...,...,...,...,...,...,...,...
70,현대차,2019,1.757339,Automobile,67,58,86,005380.KS
71,현대차,2020,1.671593,Automobile,75,64,81,005380.KS
72,현대차,2021,1.222235,Automobile,93,64,82,005380.KS
73,현대차,2022,1.381263,Automobile,83,75,50,005380.KS


### 전처리 추가된 과정

In [ ]:
# 전처리 함수 정의
def preprocess_data(df):
    # 기존 컬럼명을 사용할 수 있도록 유효성을 확인
    if 'environmental' in df.columns and 'social' in df.columns and 'governance' in df.columns:
        # ESG 영역 비중을 백분율로 환산
        df['env_percent'] = df['environmental'] / (df['environmental'] + df['social'] + df['governance'])
        df['soc_percent'] = df['social'] / (df['environmental'] + df['social'] + df['governance'])
        df['gov_percent'] = df['governance'] / (df['environmental'] + df['social'] + df['governance'])

        # 각 영역별 최종 점수 계산 (average_label 필요)
        df['env_score'] = df['average_label'] * df['env_percent']
        df['soc_score'] = df['average_label'] * df['soc_percent']
        df['gov_score'] = df['average_label'] * df['gov_percent']

        # 연도별 가중치 설정
        latest_year = df['Year'].max()
        year_weights = {
            latest_year: 0.5,
            latest_year - 1: 0.25,
            latest_year - 2: 0.125,
            latest_year - 3: 0.0625,
            latest_year - 4: 0.0625
        }

        # 가중치를 반영한 각 영역별 점수 합산
        df['environmental'] = df.apply(lambda x: x['env_score'] * year_weights.get(x['Year'], 0), axis=1)
        df['social'] = df.apply(lambda x: x['soc_score'] * year_weights.get(x['Year'], 0), axis=1)
        df['governance'] = df.apply(lambda x: x['gov_score'] * year_weights.get(x['Year'], 0), axis=1)

        # 동일 기업의 연도별 점수를 합산하여 최종 점수 도출
        final_df = df.groupby(['Company', 'industry', 'ticker']).agg({
            'environmental': 'sum',
            'social': 'sum',
            'governance': 'sum'
        }).reset_index()

        return final_df
    else:
        raise KeyError("The expected columns 'environmental', 'social', and 'governance' are not present in the dataframe.")


In [ ]:
# 데이터 전처리 실행
dummy = preprocess_data(dummy)
dummy

,Company,industry,ticker,environmental,social,governance
0,KB금융,Finance,105560.KS,1.984119,2.162083,2.484306
1,NAVER,Technology,035420.KS,1.774771,1.985935,2.164074
2,SK하이닉스,Technology,000660.KS,1.461877,1.484017,1.563182
3,기아,Automobile,000270.KS,0.794908,0.816381,0.717146
4,삼성SDI,Energy,006400.KS,1.664418,1.382815,1.600532
5,삼성물산,Construction,028260.KS,1.731279,1.292812,1.554031
6,삼성바이오로직스,Healthcare,207940.KS,0.666348,0.680800,0.772988
7,삼성생명,Finance,032830.KS,1.051869,1.291376,1.039154
8,삼성전자,Technology,005930.KS,1.803575,1.771930,1.862138
9,셀트리온,Healthcare,068270.KS,0.315728,0.324746,0.350041


### 기업 리스트 선정  함수

In [ ]:
# 기업 추천 함수
def recommend_companies(esg_weights, selected_industries, df):
    # 전처리된 데이터에서 사용자의 ESG 선호도 가중치를 반영하여 최종 점수 계산
    df['final_score'] = (
        esg_weights['environmental'] * df['environmental'] +
        esg_weights['social'] * df['social'] +
        esg_weights['governance'] * df['governance']
    )

    # 산업군 필터링
    filtered_df = df[df['industry'].isin(selected_industries)]

    # 상위 10개 기업 선정
    top_companies = filtered_df.sort_values(by='final_score', ascending=False).head(10)

    return top_companies


### 포트폴리오 비중 산출  함수

In [ ]:
# 포트폴리오 비중 계산 함수
def calculate_portfolio_weights(top_companies):
    tickers = top_companies['ticker'].tolist()
    price_data = yf.download(tickers, start="2019-01-01", end="2023-01-01")['Adj Close']

    if price_data.isnull().values.any():
        return "일부 데이터가 누락되었습니다. 다른 기업을 선택해 주세요.", None

    mu = expected_returns.mean_historical_return(price_data)
    S = risk_models.sample_cov(price_data)

    ef = EfficientFrontier(mu, S)
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    performance = ef.portfolio_performance(verbose=True)

    return cleaned_weights, performance

### 인터페이스 구현 함수

In [ ]:
# 전체 그라디오 인터페이스 함수
def gradio_interface(environmental, social, governance, industries):
    esg_weights = {'environmental': environmental, 'social': social, 'governance': governance}

    # 데이터 전처리, 단, 이미 전처리가 이미 끝난 채로 투입되므로 함수를 실행하지 않고 함수 실행 결과를 복제
    processed_df = dummy.copy()

    # 기업 추천 실행
    top_companies = recommend_companies(esg_weights, industries, processed_df)

    if top_companies.empty:
        return "선택한 조건에 맞는 기업이 없습니다.", None

    # 포트폴리오 비중 계산 실행
    portfolio_weights, portfolio_performance = calculate_portfolio_weights(top_companies)

    # 포트폴리오 비중 시각화
    fig, ax = plt.subplots()
    ax.barh(list(portfolio_weights.keys()), list(portfolio_weights.values()))
    ax.set_xlabel('Portfolio Weights')
    ax.set_title('Calculated Portfolio')

    return (
        f"Expected annual return: {portfolio_performance[0]:.2%}\n"
        f"Annual volatility: {portfolio_performance[1]:.2%}\n"
        f"Sharpe Ratio: {portfolio_performance[2]:.2f}",
        fig
    )

### 인터페이스 구성 및 실행

In [ ]:
# Gradio 인터페이스 구성
gr_interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Slider(0, 1, value=0.5, label='Environmental'),
        gr.Slider(0, 1, value=0.5, label='Social'),
        gr.Slider(0, 1, value=0.5, label='Governance'),
        gr.CheckboxGroup(choices=dummy['industry'].unique().tolist(), label="관심 산업군을 선택하세요")
    ],
    outputs=["text", "plot"],
    title="ESG 포트폴리오 최적화 시스템"
)

In [ ]:
# Launch Gradio interface
gr_interface.launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c620c0aa83ddb1f2b7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[*********************100%***********************]  10 of 10 completed


Expected annual return: 28.3%
Annual volatility: 31.5%
Sharpe Ratio: 0.84


[*********************100%***********************]  10 of 10 completed


Expected annual return: 28.3%
Annual volatility: 31.5%
Sharpe Ratio: 0.84


[*********************100%***********************]  10 of 10 completed


Expected annual return: 28.3%
Annual volatility: 31.5%
Sharpe Ratio: 0.84


[*********************100%***********************]  10 of 10 completed


Expected annual return: 28.3%
Annual volatility: 31.5%
Sharpe Ratio: 0.84


## 2. CVXOPT 라이브러리 적용

### 2.1. 기존 방법을 유지하면서 (1차로 종목 선정, ) 2차로 CVXOPT 라이브러리 적용

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt
import gradio as gr

# 전처리 함수 정의
def preprocess_data(df):
    # 기존 컬럼명을 사용할 수 있도록 유효성을 확인
    if 'environmental' in df.columns and 'social' in df.columns and 'governance' in df.columns:
        # ESG 영역 비중을 백분율로 환산
        df['env_percent'] = df['environmental'] / (df['environmental'] + df['social'] + df['governance'])
        df['soc_percent'] = df['social'] / (df['environmental'] + df['social'] + df['governance'])
        df['gov_percent'] = df['governance'] / (df['environmental'] + df['social'] + df['governance'])

        # 각 영역별 최종 점수 계산 (average_label 필요)
        df['env_score'] = df['average_label'] * df['env_percent']
        df['soc_score'] = df['average_label'] * df['soc_percent']
        df['gov_score'] = df['average_label'] * df['gov_percent']

        # 연도별 가중치 설정
        latest_year = df['Year'].max()
        year_weights = {
            latest_year: 0.5,
            latest_year - 1: 0.25,
            latest_year - 2: 0.125,
            latest_year - 3: 0.0625,
            latest_year - 4: 0.0625
        }

        # 가중치를 반영한 각 영역별 점수 합산
        df['environmental'] = df.apply(lambda x: x['env_score'] * year_weights.get(x['Year'], 0), axis=1)
        df['social'] = df.apply(lambda x: x['soc_score'] * year_weights.get(x['Year'], 0), axis=1)
        df['governance'] = df.apply(lambda x: x['gov_score'] * year_weights.get(x['Year'], 0), axis=1)

        # 동일 기업의 연도별 점수를 합산하여 최종 점수 도출
        final_df = df.groupby(['Company', 'industry', 'ticker']).agg({
            'environmental': 'sum',
            'social': 'sum',
            'governance': 'sum'
        }).reset_index()

        return final_df
    else:
        raise KeyError("The expected columns 'environmental', 'social', and 'governance' are not present in the dataframe.")

# 데이터 전처리 실행
dummy = pd.read_csv('/content/drive/MyDrive/Kwargs/240820_final_dummy.csv', encoding='euc-kr')
dummy = preprocess_data(dummy)

# 기업 추천 함수
def recommend_companies(esg_weights, selected_industries, df):
    # 전처리된 데이터에서 사용자의 ESG 선호도 가중치를 반영하여 최종 점수 계산
    df['final_score'] = (
        esg_weights['environmental'] * df['environmental'] +
        esg_weights['social'] * df['social'] +
        esg_weights['governance'] * df['governance']
    )

    # 산업군 필터링
    filtered_df = df[df['industry'].isin(selected_industries)]

    # 상위 10개 기업 선정
    top_companies = filtered_df.sort_values(by='final_score', ascending=False).head(10)

    return top_companies

# 포트폴리오 비중 계산 함수 with CVXOPT
def calculate_portfolio_weights(top_companies):
    tickers = top_companies['ticker'].tolist()
    price_data = yf.download(tickers, start="2019-01-01", end="2023-01-01")['Adj Close']

    if price_data.isnull().values.any():
        return "일부 데이터가 누락되었습니다. 다른 기업을 선택해 주세요.", None

    # 일별 수익률 계산
    returns = price_data.pct_change().dropna()

    # 평균 수익률과 공분산 행렬
    mu = returns.mean().values
    Sigma = returns.cov().values

    # `cvxopt`에서 사용할 행렬 형식으로 변환
    n = len(mu)
    P = matrix(Sigma)
    q = matrix(np.zeros(n))
    G = matrix(-np.eye(n))
    h = matrix(np.zeros(n))
    A = matrix(1.0, (1, n))
    b = matrix(1.0)

    # 쿼드라틱 프로그래밍 솔버 실행
    sol = solvers.qp(P, q, G, h, A, b)

    # 최적 가중치 추출
    weights = np.array(sol['x']).flatten()

    # 포트폴리오 성과 지표 계산
    expected_return = np.dot(weights, mu)
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(Sigma, weights)))
    sharpe_ratio = expected_return / expected_volatility

    # 가중치 정리
    cleaned_weights = dict(zip(tickers, weights))

    return cleaned_weights, (expected_return, expected_volatility, sharpe_ratio)

# 인터페이스 구현 함수
def gradio_interface(environmental, social, governance, industries):
    esg_weights = {'environmental': environmental, 'social': social, 'governance': governance}

    # 데이터 전처리, 단, 이미 전처리가 이미 끝난 채로 투입되므로 함수를 실행하지 않고 함수 실행 결과를 복제
    processed_df = dummy.copy()

    # 기업 추천 실행
    top_companies = recommend_companies(esg_weights, industries, processed_df)

    if top_companies.empty:
        return "선택한 조건에 맞는 기업이 없습니다.", None

    # 포트폴리오 비중 계산 실행
    portfolio_weights, portfolio_performance = calculate_portfolio_weights(top_companies)

    # 포트폴리오 비중 시각화
    fig, ax = plt.subplots()
    ax.barh(list(portfolio_weights.keys()), list(portfolio_weights.values()))
    ax.set_xlabel('Portfolio Weights')
    ax.set_title('Calculated Portfolio')

    return (
        f"Expected annual return: {portfolio_performance[0]:.2%}\n"
        f"Annual volatility: {portfolio_performance[1]:.2%}\n"
        f"Sharpe Ratio: {portfolio_performance[2]:.2f}",
        fig
    )

# Gradio 인터페이스 구성
gr_interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Slider(0, 1, value=0.5, label='Environmental'),
        gr.Slider(0, 1, value=0.5, label='Social'),
        gr.Slider(0, 1, value=0.5, label='Governance'),
        gr.CheckboxGroup(choices=dummy['industry'].unique().tolist(), label="관심 산업군을 선택하세요")
    ],
    outputs=["text", "plot"],
    title="ESG 포트폴리오 최적화 시스템"
)

# Launch Gradio interface
gr_interface.launch(share=True, debug=True)


### 2.2. CVXOPT 방법을 적용하되,  ESG 관련 피쳐를 함수 내부 파라미터로 사용

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt
import gradio as gr

# 전처리 함수 정의
def preprocess_data(df):
    # 기존 컬럼명을 사용할 수 있도록 유효성을 확인
    if 'environmental' in df.columns and 'social' in df.columns and 'governance' in df.columns:
        # ESG 영역 비중을 백분율로 환산
        df['env_percent'] = df['environmental'] / (df['environmental'] + df['social'] + df['governance'])
        df['soc_percent'] = df['social'] / (df['environmental'] + df['social'] + df['governance'])
        df['gov_percent'] = df['governance'] / (df['environmental'] + df['social'] + df['governance'])

        # 각 영역별 최종 점수 계산 (average_label 필요)
        df['env_score'] = df['average_label'] * df['env_percent']
        df['soc_score'] = df['average_label'] * df['soc_percent']
        df['gov_score'] = df['average_label'] * df['gov_percent']

        # 연도별 가중치 설정
        latest_year = df['Year'].max()
        year_weights = {
            latest_year: 0.5,
            latest_year - 1: 0.25,
            latest_year - 2: 0.125,
            latest_year - 3: 0.0625,
            latest_year - 4: 0.0625
        }

        # 가중치를 반영한 각 영역별 점수 합산
        df['environmental'] = df.apply(lambda x: x['env_score'] * year_weights.get(x['Year'], 0), axis=1)
        df['social'] = df.apply(lambda x: x['soc_score'] * year_weights.get(x['Year'], 0), axis=1)
        df['governance'] = df.apply(lambda x: x['gov_score'] * year_weights.get(x['Year'], 0), axis=1)

        # 동일 기업의 연도별 점수를 합산하여 최종 점수 도출
        final_df = df.groupby(['Company', 'industry', 'ticker']).agg({
            'environmental': 'sum',
            'social': 'sum',
            'governance': 'sum'
        }).reset_index()

        return final_df
    else:
        raise KeyError("The expected columns 'environmental', 'social', and 'governance' are not present in the dataframe.")

# 데이터 전처리 실행
dummy = pd.read_csv('/content/drive/MyDrive/Kwargs/240820_final_dummy.csv', encoding='euc-kr')
dummy = preprocess_data(dummy)

# 포트폴리오 비중 계산 함수 with CVXOPT
def calculate_portfolio_weights(df, esg_weights):
    tickers = df['ticker'].tolist()
    price_data = yf.download(tickers, start="2019-01-01", end="2023-01-01")['Adj Close']

    if price_data.isnull().values.any():
        return "일부 데이터가 누락되었습니다. 다른 기업을 선택해 주세요.", None

    # 일별 수익률 계산
    returns = price_data.pct_change().dropna()

    # 평균 수익률과 공분산 행렬
    mu = returns.mean().values
    Sigma = returns.cov().values

    # 사용자 선호도를 반영한 ESG 점수 조정
    df['final_esg_score'] = (
        esg_weights['environmental'] * df['environmental'] +
        esg_weights['social'] * df['social'] +
        esg_weights['governance'] * df['governance']
    )

    # 기대 수익률을 ESG 점수로 조정 (가중 평균 반영)
    adjusted_returns = mu * (1 + df['final_esg_score'].values)

    # ESG 이슈 빈도를 반영한 위험도 조정 (예: ESG 이슈가 클수록 위험도 증가)
    adjusted_cov_matrix = Sigma.copy()
    for i, company in enumerate(tickers):
        issue_risk_factor = df.loc[df['ticker'] == company, 'final_esg_score'].values[0]
        adjusted_cov_matrix[i, i] += issue_risk_factor  # 리스크 증가 반영

    # `cvxopt`에서 사용할 행렬 형식으로 변환
    n = len(mu)
    P = matrix(adjusted_cov_matrix)
    q = matrix(-adjusted_returns)  # 조정된 기대 수익률 사용
    G = matrix(-np.eye(n))
    h = matrix(np.zeros(n))
    A = matrix(1.0, (1, n))
    b = matrix(1.0)

    # 쿼드라틱 프로그래밍 솔버 실행
    sol = solvers.qp(P, q, G, h, A, b)

    # 최적 가중치 추출
    weights = np.array(sol['x']).flatten()

    # 포트폴리오 성과 지표 계산
    expected_return = np.dot(weights, mu)
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(Sigma, weights)))
    sharpe_ratio = expected_return / expected_volatility

    # 가중치 정리
    cleaned_weights = dict(zip(tickers, weights))

    return cleaned_weights, (expected_return, expected_volatility, sharpe_ratio)

# 인터페이스 구현 함수
def gradio_interface(environmental, social, governance, industries):
    esg_weights = {'environmental': environmental, 'social': social, 'governance': governance}

    # 데이터 전처리, 단, 이미 전처리가 이미 끝난 채로 투입되므로 함수를 실행하지 않고 함수 실행 결과를 복제
    processed_df = dummy[dummy['industry'].isin(industries)].copy()

    # 포트폴리오 비중 계산 실행
    portfolio_weights, portfolio_performance = calculate_portfolio_weights(processed_df, esg_weights)

    # 포트폴리오 비중 시각화
    fig, ax = plt.subplots()
    ax.barh(list(portfolio_weights.keys()), list(portfolio_weights.values()))
    ax.set_xlabel('Portfolio Weights')
    ax.set_title('Calculated Portfolio')

    return (
        f"Expected annual return: {portfolio_performance[0]:.2%}\n"
        f"Annual volatility: {portfolio_performance[1]:.2%}\n"
        f"Sharpe Ratio: {portfolio_performance[2]:.2f}",
        fig
    )

# Gradio 인터페이스 구성
gr_interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Slider(0, 1, value=0.5, label='Environmental'),
        gr.Slider(0, 1, value=0.5, label='Social'),
        gr.Slider(0, 1, value=0.5, label='Governance'),
        gr.CheckboxGroup(choices=dummy['industry'].unique().tolist(), label="관심 산업군을 선택하세요")
    ],
    outputs=["text", "plot"],
    title="ESG 포트폴리오 최적화 시스템"
)

# Launch Gradio interface
gr_interface.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e8701d13bbf093cb07.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[*********************100%***********************]  15 of 15 completed


     pcost       dcost       gap    pres   dres
 0:  4.6873e-02 -9.9641e-01  1e+00  2e-16  4e+00
 1:  4.6002e-02 -1.5301e-03  5e-02  4e-17  2e-01
 2:  4.3401e-02  4.0076e-02  3e-03  3e-17  6e-17
 3:  4.3365e-02  4.3324e-02  4e-05  7e-17  5e-17
 4:  4.3365e-02  4.3365e-02  4e-07  5e-17  3e-17
 5:  4.3365e-02  4.3365e-02  4e-09  5e-17  5e-17
Optimal solution found.


[*********************100%***********************]  15 of 15 completed


     pcost       dcost       gap    pres   dres
 0:  6.2741e-02 -9.8793e-01  1e+00  0e+00  4e+00
 1:  6.1644e-02  9.7890e-03  5e-02  8e-17  2e-01
 2:  5.9293e-02  5.6560e-02  3e-03  2e-16  7e-04
 3:  5.9273e-02  5.9243e-02  3e-05  2e-16  7e-06
 4:  5.9273e-02  5.9272e-02  3e-07  5e-17  7e-08
 5:  5.9273e-02  5.9273e-02  3e-09  1e-16  7e-10
Optimal solution found.


### w3

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import gradio as gr

# 전처리 함수 정의
def preprocess_data(df):
    # 기존 컬럼명을 사용할 수 있도록 유효성을 확인
    if 'environmental' in df.columns and 'social' in df.columns and 'governance' in df.columns:
        # ESG 영역 비중을 백분율로 환산
        df['env_percent'] = df['environmental'] / (df['environmental'] + df['social'] + df['governance'])
        df['soc_percent'] = df['social'] / (df['environmental'] + df['social'] + df['governance'])
        df['gov_percent'] = df['governance'] / (df['environmental'] + df['social'] + df['governance'])

        # 각 영역별 최종 점수 계산 (average_label 필요)
        df['env_score'] = df['average_label'] * df['env_percent']
        df['soc_score'] = df['average_label'] * df['soc_percent']
        df['gov_score'] = df['average_label'] * df['gov_percent']

        # 연도별 가중치 설정
        latest_year = df['Year'].max()
        year_weights = {
            latest_year: 0.5,
            latest_year - 1: 0.25,
            latest_year - 2: 0.125,
            latest_year - 3: 0.0625,
            latest_year - 4: 0.0625
        }

        # 가중치를 반영한 각 영역별 점수 합산
        df['environmental'] = df.apply(lambda x: x['env_score'] * year_weights.get(x['Year'], 0), axis=1)
        df['social'] = df.apply(lambda x: x['soc_score'] * year_weights.get(x['Year'], 0), axis=1)
        df['governance'] = df.apply(lambda x: x['gov_score'] * year_weights.get(x['Year'], 0), axis=1)

        # 동일 기업의 연도별 점수를 합산하여 최종 점수 도출
        final_df = df.groupby(['Company', 'industry', 'ticker']).agg({
            'environmental': 'sum',
            'social': 'sum',
            'governance': 'sum'
        }).reset_index()

        return final_df
    else:
        raise KeyError("The expected columns 'environmental', 'social', and 'governance' are not present in the dataframe.")

# 데이터 전처리 실행
dummy = pd.read_csv('/content/drive/MyDrive/Kwargs/240820_final_dummy.csv', encoding='euc-kr')
dummy = preprocess_data(dummy)

# 기업 추천 함수
def recommend_companies(esg_weights, selected_industries, df):
    # 전처리된 데이터에서 사용자의 ESG 선호도 가중치를 반영하여 최종 점수 계산
    df['final_score'] = (
        esg_weights['environmental'] * df['environmental'] +
        esg_weights['social'] * df['social'] +
        esg_weights['governance'] * df['governance']
    )

    # 산업군 필터링
    filtered_df = df[df['industry'].isin(selected_industries)]

    # 상위 10개 기업 선정
    top_companies = filtered_df.sort_values(by='final_score', ascending=False).head(10)

    return top_companies

# 포트폴리오 최적화 함수 (MPT 적용)
def calculate_portfolio_weights(top_companies):
    tickers = top_companies['ticker'].tolist()
    price_data = yf.download(tickers, start="2019-01-01", end="2023-01-01")['Adj Close']

    if price_data.isnull().values.any():
        return "일부 데이터가 누락되었습니다. 다른 기업을 선택해 주세요.", None

    # 일별 수익률 계산
    returns = price_data.pct_change().dropna()

    # 평균 수익률과 공분산 행렬
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    num_assets = len(tickers)
    risk_free_rate = 0.01

    # 포트폴리오의 성과를 계산하는 함수
    def portfolio_performance(weights, mean_returns, cov_matrix):
        returns = np.sum(mean_returns * weights) * 252
        std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
        return std_dev, returns

    # 샤프 비율을 최대화하는 목표 함수
    def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
        p_var, p_ret = portfolio_performance(weights, mean_returns, cov_matrix)
        return -(p_ret - risk_free_rate) / p_var

    # 모든 자산의 가중치 합이 1이 되도록 제약 조건 설정
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for asset in range(num_assets))
    initial_weights = num_assets * [1. / num_assets,]

    # 최적화 실행
    result = minimize(neg_sharpe_ratio, initial_weights, args=(mean_returns, cov_matrix, risk_free_rate),
                      method='SLSQP', bounds=bounds, constraints=constraints)

    # 최적의 가중치와 포트폴리오 성과 계산
    optimal_weights = result.x
    portfolio_std_dev, portfolio_return = portfolio_performance(optimal_weights, mean_returns, cov_matrix)
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev

    cleaned_weights = dict(zip(tickers, optimal_weights))

    return cleaned_weights, (portfolio_return, portfolio_std_dev, sharpe_ratio)

# 인터페이스 구현 함수
def gradio_interface(environmental, social, governance, industries):
    esg_weights = {'environmental': environmental, 'social': social, 'governance': governance}

    # 데이터 전처리, 단, 이미 전처리가 이미 끝난 채로 투입되므로 함수를 실행하지 않고 함수 실행 결과를 복제
    processed_df = dummy.copy()

    # 기업 추천 실행
    top_companies = recommend_companies(esg_weights, industries, processed_df)

    if top_companies.empty:
        return "선택한 조건에 맞는 기업이 없습니다.", None

    # 포트폴리오 비중 계산 실행
    portfolio_weights, portfolio_performance = calculate_portfolio_weights(top_companies)

    # 포트폴리오 비중 시각화
    fig, ax = plt.subplots()
    ax.barh(list(portfolio_weights.keys()), list(portfolio_weights.values()))
    ax.set_xlabel('Portfolio Weights')
    ax.set_title('Calculated Portfolio')

    return (
        f"Expected annual return: {portfolio_performance[0]:.2%}\n"
        f"Annual volatility: {portfolio_performance[1]:.2%}\n"
        f"Sharpe Ratio: {portfolio_performance[2]:.2f}",
        fig
    )

# Gradio 인터페이스 구성
gr_interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Slider(0, 1, value=0.5, label='Environmental'),
        gr.Slider(0, 1, value=0.5, label='Social'),
        gr.Slider(0, 1, value=0.5, label='Governance'),
        gr.CheckboxGroup(choices=dummy['industry'].unique().tolist(), label="관심 산업군을 선택하세요")
    ],
    outputs=["text", "plot"],
    title="ESG 포트폴리오 최적화 시스템"
)

# Launch Gradio interface
gr_interface.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://9fd7d84c49849f9e44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed


### Black_Litterman method

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt
import gradio as gr
from pypfopt import risk_models, BlackLittermanModel, expected_returns

# 전처리 함수 정의
def preprocess_data(df):
    # 기존 컬럼명을 사용할 수 있도록 유효성을 확인
    if 'environmental' in df.columns and 'social' in df.columns and 'governance' in df.columns:
        # ESG 영역 비중을 백분율로 환산
        df['env_percent'] = df['environmental'] / (df['environmental'] + df['social'] + df['governance'])
        df['soc_percent'] = df['social'] / (df['environmental'] + df['social'] + df['governance'])
        df['gov_percent'] = df['governance'] / (df['environmental'] + df['social'] + df['governance'])

        # 각 영역별 최종 점수 계산 (average_label 필요)
        df['env_score'] = df['average_label'] * df['env_percent']
        df['soc_score'] = df['average_label'] * df['soc_percent']
        df['gov_score'] = df['average_label'] * df['gov_percent']

        # 연도별 가중치 설정
        latest_year = df['Year'].max()
        year_weights = {
            latest_year: 0.5,
            latest_year - 1: 0.25,
            latest_year - 2: 0.125,
            latest_year - 3: 0.0625,
            latest_year - 4: 0.0625
        }

        # 가중치를 반영한 각 영역별 점수 합산
        df['environmental'] = df.apply(lambda x: x['env_score'] * year_weights.get(x['Year'], 0), axis=1)
        df['social'] = df.apply(lambda x: x['soc_score'] * year_weights.get(x['Year'], 0), axis=1)
        df['governance'] = df.apply(lambda x: x['gov_score'] * year_weights.get(x['Year'], 0), axis=1)

        # 동일 기업의 연도별 점수를 합산하여 최종 점수 도출
        final_df = df.groupby(['Company', 'industry', 'ticker']).agg({
            'environmental': 'sum',
            'social': 'sum',
            'governance': 'sum'
        }).reset_index()

        return final_df
    else:
        raise KeyError("The expected columns 'environmental', 'social', and 'governance' are not present in the dataframe.")

# 데이터 전처리 실행
dummy = pd.read_csv('/content/drive/MyDrive/Kwargs/240820_final_dummy.csv', encoding='euc-kr')
dummy = preprocess_data(dummy)

# 포트폴리오 비중 계산 함수 with Black-Litterman
def calculate_portfolio_weights(df, esg_weights):
    tickers = df['ticker'].tolist()
    price_data = yf.download(tickers, start="2019-01-01", end="2023-01-01")['Adj Close']

    if price_data.isnull().values.any():
        return "일부 데이터가 누락되었습니다. 다른 기업을 선택해 주세요.", None

    # 평균 수익률과 공분산 행렬
    mu_market = expected_returns.capm_return(price_data)  # CAPM을 통한 시장 균형 수익률 계산
    Sigma = risk_models.sample_cov(price_data)  # 샘플 공분산 행렬

    # 사용자 선호도를 반영한 ESG 점수 조정
    df['final_esg_score'] = (
        esg_weights['environmental'] * df['environmental'] +
        esg_weights['social'] * df['social'] +
        esg_weights['governance'] * df['governance']
    )

    # 사용자 ESG 점수를 투자자의 견해로 반영
    # 예: ESG 점수가 높을수록 해당 기업의 기대 수익률이 증가
    P = np.eye(len(tickers))  # P 매트릭스: 각 자산에 대해 의견을 반영
    Q = df['final_esg_score'].values  # Q 벡터: 각 자산에 대한 투자자의 의견 (ESG 점수 반영)

    # Black-Litterman 모델 적용
    bl = BlackLittermanModel(Sigma, pi=mu_market, P=P, Q=Q)
    adjusted_returns = bl.bl_returns()

    # `cvxopt`에서 사용할 행렬 형식으로 변환
    n = len(mu_market)
    P_opt = matrix(Sigma.values)
    q_opt = matrix(-adjusted_returns.values)
    G = matrix(-np.eye(n))
    h = matrix(np.zeros(n))
    A = matrix(1.0, (1, n))
    b = matrix(1.0)

    # 쿼드라틱 프로그래밍 솔버 실행
    sol = solvers.qp(P_opt, q_opt, G, h, A, b)

    # 최적 가중치 추출
    weights = np.array(sol['x']).flatten()

    # 포트폴리오 성과 지표 계산
    expected_return = np.dot(weights, mu_market)
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(Sigma.values, weights)))
    sharpe_ratio = expected_return / expected_volatility

    # 가중치 정리
    cleaned_weights = dict(zip(tickers, weights))

    return cleaned_weights, (expected_return, expected_volatility, sharpe_ratio)

# 인터페이스 구현 함수
def gradio_interface(environmental, social, governance, industries):
    esg_weights = {'environmental': environmental, 'social': social, 'governance': governance}

    # 데이터 전처리, 단, 이미 전처리가 이미 끝난 채로 투입되므로 함수를 실행하지 않고 함수 실행 결과를 복제
    processed_df = dummy[dummy['industry'].isin(industries)].copy()

    # 포트폴리오 비중 계산 실행
    portfolio_weights, portfolio_performance = calculate_portfolio_weights(processed_df, esg_weights)

    # 포트폴리오 비중 시각화
    fig, ax = plt.subplots()
    ax.barh(list(portfolio_weights.keys()), list(portfolio_weights.values()))
    ax.set_xlabel('Portfolio Weights')
    ax.set_title('Calculated Portfolio')

    return (
        f"Expected annual return: {portfolio_performance[0]:.2%}\n"
        f"Annual volatility: {portfolio_performance[1]:.2%}\n"
        f"Sharpe Ratio: {portfolio_performance[2]:.2f}",
        fig
    )

# Gradio 인터페이스 구성
gr_interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Slider(0, 1, value=0.5, label='Environmental'),
        gr.Slider(0, 1, value=0.5, label='Social'),
        gr.Slider(0, 1, value=0.5, label='Governance'),
        gr.CheckboxGroup(choices=dummy['industry'].unique().tolist(), label="관심 산업군을 선택하세요")
    ],
    outputs=["text", "plot"],
    title="ESG 포트폴리오 최적화 시스템"
)

# Launch Gradio interface
gr_interface.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://73fc0f6d66b89aeaf3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[*********************100%***********************]  5 of 5 completed


     pcost       dcost       gap    pres   dres
 0: -1.5259e+00 -2.5702e+00  9e+00  3e+00  1e+00
 1: -1.3604e+00 -2.2415e+00  9e-01  2e-15  3e-16
 2: -1.4159e+00 -1.5744e+00  2e-01  4e-16  2e-16
 3: -1.5499e+00 -1.5780e+00  3e-02  2e-17  9e-17
 4: -1.5591e+00 -1.5595e+00  4e-04  2e-16  9e-17
 5: -1.5593e+00 -1.5593e+00  4e-06  1e-16  1e-16
 6: -1.5593e+00 -1.5593e+00  4e-08  1e-16  7e-17
Optimal solution found.


[*********************100%***********************]  15 of 15 completed


     pcost       dcost       gap    pres   dres
 0: -3.7650e+00 -3.6579e+00  4e+01  6e+00  8e-01
 1: -2.1833e+00 -3.4664e+00  1e+00  3e-15  4e-16
 2: -2.3099e+00 -2.6173e+00  3e-01  9e-16  4e-16
 3: -2.5354e+00 -2.6344e+00  1e-01  2e-16  2e-16
 4: -2.5788e+00 -2.5843e+00  6e-03  2e-18  2e-16
 5: -2.5816e+00 -2.5817e+00  6e-05  1e-16  3e-16
 6: -2.5817e+00 -2.5817e+00  6e-07  2e-16  2e-16
Optimal solution found.


[*********************100%***********************]  15 of 15 completed


     pcost       dcost       gap    pres   dres
 0: -4.6574e+00 -4.0516e+00  4e+01  6e+00  8e-01
 1: -2.5051e+00 -3.8361e+00  1e+00  3e-15  4e-16
 2: -2.6734e+00 -2.9883e+00  3e-01  4e-16  3e-16
 3: -2.9149e+00 -2.9917e+00  8e-02  2e-16  2e-16
 4: -2.9551e+00 -2.9585e+00  3e-03  2e-16  2e-16
 5: -2.9570e+00 -2.9570e+00  3e-05  2e-16  2e-16
 6: -2.9570e+00 -2.9570e+00  3e-07  1e-21  2e-16
Optimal solution found.


## 부족한 데이터 생성 및 라벨링 (이제 필요 없음)

이 아래 데이터셋 하드코딩 부분은 지금은 실행하되,이 노트북에서는 일단 데이터셋에 포함된 컬럼이 없으니 임의로 컬럼을 새로 부여한것임.실제 데이터셋에는 처음부터 포함할 예정.

In [ ]:
# Step 2: Define industry and ticker mapping (데이터셋에 이미 포함되어 있다고 가정)
# 데이터셋에 기업명에 해당하는 티커와 산업군, ESG 점수가 미리 포함되어 있다고 가정합니다.

In [ ]:
# 각사별 산업군 라벨 부여
industry_mapping = {
    'KB금융': 'Finance',
    'NAVER': 'Technology',
    'SK하이닉스': 'Technology',
    '기아': 'Automobile',
    '삼성SDI': 'Energy',
    '삼성물산': 'Construction',
    '삼성바이오로직스': 'Healthcare',
    '삼성생명': 'Finance',
    '삼성전자': 'Technology',
    '셀트리온': 'Healthcare',
    '신한지주': 'Finance',
    '카카오': 'Technology',
    '포스코 홀딩스': 'Materials',
    '현대모비스': 'Automobile',
    '현대차': 'Automobile'
}

esg_data['industry'] = esg_data['Company'].map(industry_mapping)

# ESG 각사별 기사화 비중 라벨 임의 생성
np.random.seed(42)  # For reproducibility
esg_data['environmental'] = np.random.randint(50, 100, size=len(esg_data))
esg_data['social'] = np.random.randint(50, 100, size=len(esg_data))
esg_data['governance'] = np.random.randint(50, 100, size=len(esg_data))


# 야후파이낸셜 기준 티커(종목코드) 라벨 생성
ticker_mapping = {
    'KB금융': '105560.KS',
    'NAVER': '035420.KS',
    'SK하이닉스': '000660.KS',
    '기아': '000270.KS',
    '삼성SDI': '006400.KS',
    '삼성물산': '028260.KS',
    '삼성바이오로직스': '207940.KS',
    '삼성생명': '032830.KS',
    '삼성전자': '005930.KS',
    '셀트리온': '068270.KS',
    '신한지주': '055550.KS',
    '카카오': '035720.KS',
    '포스코 홀딩스': '005490.KS',
    '현대모비스': '012330.KS',
    '현대차': '005380.KS'
}

# Add ticker information to the dataframe
esg_data['ticker'] = esg_data['Company'].map(ticker_mapping)



In [ ]:
esg_data

,Company,Year,average_label,industry,environmental,social,governance,ticker
0,KB금융,2019,5.016282,Finance,88,85,62,105560.KS
1,KB금융,2020,5.109671,Finance,78,63,81,105560.KS
2,KB금융,2021,5.996064,Finance,64,80,88,105560.KS
3,KB금융,2022,6.966534,Finance,92,97,98,105560.KS
4,KB금융,2023,7.012989,Finance,57,64,81,105560.KS
...,...,...,...,...,...,...,...,...
70,현대차,2019,1.757339,Automobile,67,58,86,005380.KS
71,현대차,2020,1.671593,Automobile,75,64,81,005380.KS
72,현대차,2021,1.222235,Automobile,93,64,82,005380.KS
73,현대차,2022,1.381263,Automobile,83,75,50,005380.KS


In [ ]:
esg_data.to_csv('/content/drive/MyDrive/Kwargs/240820_final_dummy.csv', encoding='euc-kr', index=False)

In [ ]:
# Step 1: Load the provided dataset
# 이미 완성된 데이터셋을 로드합니다. 여기서는 파일 경로를 넣어주면 됩니다.
file_path = '/content/drive/MyDrive/Kwargs/모델B1/결과/average_predictions.csv'  # Replace with your file path
esg_data = pd.read_csv(file_path)

In [ ]:
esg_data

,Company,Year,average_label
0,KB금융,2019,5.016282
1,KB금융,2020,5.109671
2,KB금융,2021,5.996064
3,KB금융,2022,6.966534
4,KB금융,2023,7.012989
...,...,...,...
70,현대차,2019,1.757339
71,현대차,2020,1.671593
72,현대차,2021,1.222235
73,현대차,2022,1.381263


## TEST

In [ ]:
import gradio as gr
import pandas as pd

데이터셋

In [ ]:
# Sample data
data = [
    {'company': 'SK하이닉스', 'industry': 'Technology', 'environmental': 70, 'social': 75, 'governance': 80},
    {'company': '셀트리온', 'industry': 'Healthcare', 'environmental': 65, 'social': 70, 'governance': 75},
    {'company': 'LG에너지솔루션', 'industry': 'Energy', 'environmental': 60, 'social': 65, 'governance': 70},
    {'company': '한국투자증권', 'industry': 'Finance', 'environmental': 75, 'social': 80, 'governance': 85},
    {'company': '삼성전자', 'industry': 'Technology', 'environmental': 80, 'social': 85, 'governance': 90},
    {'company': '아모레퍼시픽', 'industry': 'Healthcare', 'environmental': 55, 'social': 60, 'governance': 65},
    {'company': '포스코케미칼', 'industry': 'Energy', 'environmental': 50, 'social': 55, 'governance': 60},
    {'company': 'KB국민은행', 'industry': 'Finance', 'environmental': 85, 'social': 90, 'governance': 95},
]

df = pd.DataFrame(data)

추천 알고리즘 정의

In [ ]:
def recommend_portfolio(tech, health, energy, finance, env, soc, gov):
    industry_interest = {
        'Technology': tech,
        'Healthcare': health,
        'Energy': energy,
        'Finance': finance
    }

    esg_interest = {
        'Environmental': env,
        'Social': soc,
        'Governance': gov
    }

    def calculate_weighted_score(company):
        industry_weight = industry_interest.get(company['industry'], 0)
        esg_score = (
            company['environmental'] * esg_interest['Environmental'] +
            company['social'] * esg_interest['Social'] +
            company['governance'] * esg_interest['Governance']
        )
        return industry_weight * esg_score

    df['weighted_score'] = df.apply(calculate_weighted_score, axis=1)
    sorted_companies = df.sort_values(by='weighted_score', ascending=False)

    return sorted_companies[['company', 'weighted_score']]



입력값 정의

In [ ]:
inputs = [
    gr.Slider(0, 1, value=0.5, label='Technology'),
    gr.Slider(0, 1, value=0.5, label='Healthcare'),
    gr.Slider(0, 1, value=0.5, label='Energy'),
    gr.Slider(0, 1, value=0.5, label='Finance'),
    gr.Slider(0, 1, value=0.5, label='Environmental'),
    gr.Slider(0, 1, value=0.5, label='Social'),
    gr.Slider(0, 1, value=0.5, label='Governance')
]

### Gradio 앱 실행

In [ ]:
outputs = gr.DataFrame()

demo = gr.Interface(fn=recommend_portfolio, inputs=inputs, outputs=outputs, title='ESG 포트폴리오 추천 시스템')

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b7d2576960c9865827.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### 포트폴리오 편성 알고리즘  (임의작성된 알고리즘)

In [ ]:


# Step 3: Define the recommendation algorithm
def recommend_portfolio_updated(technology, healthcare, energy, finance, automobile, materials, construction, env, soc, gov):
    industry_interest = {
        'Technology': technology,
        'Healthcare': healthcare,
        'Energy': energy,
        'Finance': finance,
        'Automobile': automobile,
        'Materials': materials,
        'Construction': construction,
    }

    esg_interest = {
        'Environmental': env,
        'Social': soc,
        'Governance': gov
    }

    def calculate_weighted_score(company):
        industry_weight = industry_interest.get(company['industry'], 0)
        esg_score = (
            company['environmental'] * esg_interest['Environmental'] +
            company['social'] * esg_interest['Social'] +
            company['governance'] * esg_interest['Governance']
        )
        return industry_weight * esg_score

    esg_data['weighted_score'] = esg_data.apply(calculate_weighted_score, axis=1)
    sorted_companies = esg_data.sort_values(by='weighted_score', ascending=False)

    return sorted_companies[['company', 'industry', 'weighted_score']]

# Step 4: Define Gradio inputs and outputs with checkboxes for industries
inputs = [
    gr.Checkbox(label='Technology', value=True),
    gr.Checkbox(label='Healthcare', value=True),
    gr.Checkbox(label='Energy', value=True),
    gr.Checkbox(label='Finance', value=True),
    gr.Checkbox(label='Automobile', value=True),
    gr.Checkbox(label='Materials', value=True),
    gr.Checkbox(label='Construction', value=True),
    gr.Slider(0, 1, value=0.5, label='Environmental'),
    gr.Slider(0, 1, value=0.5, label='Social'),
    gr.Slider(0, 1, value=0.5, label='Governance')
]

outputs = gr.DataFrame()



NameError: name 'gr' is not defined

### 포트폴리오 편성 알고리즘(PyPortfolioOpt 기반)


In [ ]:
# Step 4: Fetch historical price data for these companies
def fetch_price_data(companies):
    price_data = yf.download(companies, start="2019-01-01", end="2023-01-01")['Adj Close']
    return price_data

In [ ]:
# Step 5: Create and launch the Gradio interface
demo = gr.Interface(
    fn=recommend_portfolio_updated,
    inputs=inputs,
    outputs=outputs,
    title='ESG 포트폴리오 추천 시스템'
)

# Run this line in your local environment to launch the app



In [ ]:
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5bbeb245941476338d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### Streamlit

In [ ]:
!pip install streamlit pyngrok pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
# Create a streamlit_app.py file
with open('streamlit_app.py', 'w') as f:
    f.write('''
import streamlit as st
import pandas as pd

# Sample data
data = [
    {'company': 'Company A', 'industry': 'Technology', 'environmental': 70, 'social': 75, 'governance': 80},
    {'company': 'Company B', 'industry': 'Healthcare', 'environmental': 65, 'social': 70, 'governance': 75},
    {'company': 'Company C', 'industry': 'Energy', 'environmental': 60, 'social': 65, 'governance': 70},
    {'company': 'Company D', 'industry': 'Finance', 'environmental': 75, 'social': 80, 'governance': 85},
    {'company': 'Company E', 'industry': 'Technology', 'environmental': 80, 'social': 85, 'governance': 90},
    {'company': 'Company F', 'industry': 'Healthcare', 'environmental': 55, 'social': 60, 'governance': 65},
    {'company': 'Company G', 'industry': 'Energy', 'environmental': 50, 'social': 55, 'governance': 60},
    {'company': 'Company H', 'industry': 'Finance', 'environmental': 85, 'social': 90, 'governance': 95},
]

df = pd.DataFrame(data)

st.title('ESG 포트폴리오 추천 시스템')

st.sidebar.header('사용자 입력 데이터')
industry_interest = {
    'Technology': st.sidebar.slider('Technology', 0.0, 1.0, 0.3),
    'Healthcare': st.sidebar.slider('Healthcare', 0.0, 1.0, 0.2),
    'Energy': st.sidebar.slider('Energy', 0.0, 1.0, 0.1),
    'Finance': st.sidebar.slider('Finance', 0.0, 1.0, 0.4)
}

esg_interest = {
    'Environmental': st.sidebar.slider('Environmental', 0.0, 1.0, 0.4),
    'Social': st.sidebar.slider('Social', 0.0, 1.0, 0.3),
    'Governance': st.sidebar.slider('Governance', 0.0, 1.0, 0.3)
}

def calculate_weighted_score(company):
    industry_weight = industry_interest.get(company['industry'], 0)
    esg_score = (
        company['environmental'] * esg_interest['Environmental'] +
        company['social'] * esg_interest['Social'] +
        company['governance'] * esg_interest['Governance']
    )
    return industry_weight * esg_score

df['weighted_score'] = df.apply(calculate_weighted_score, axis=1)
sorted_companies = df.sort_values(by='weighted_score', ascending=False)

st.subheader('추천 포트폴리오')
st.write(sorted_companies[['company', 'weighted_score']])
''')


In [ ]:
import os
os.environ["NGROK_AUTH_TOKEN"] = "2TN2r2f7OikNCdHhLVAiOScGk2N_6ZG2G8XAtGWa3hzuP8o2X"

In [ ]:
from pyngrok import ngrok

# Set up a tunnel
ngrok.set_auth_token(os.environ["NGROK_AUTH_TOKEN"])
public_url = ngrok.connect(8501, bind_tls=True)
print(f"Streamlit app is live at: {public_url}")

# Run Streamlit app
!streamlit run streamlit_app.py &

Streamlit app is live at: NgrokTunnel: "https://f658-34-86-85-194.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.85.194:8501

  Stopping...
